In [53]:
import re
import json
import pytz
import time
import requests
import random
from fake_useragent import UserAgent
from celery import Celery
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import rethinkdb as r
from pymongo import MongoClient
from bson.objectid import ObjectId

In [ ]:
profile = {}

In [123]:
def random_header(url):
    ua = UserAgent()
    random_header = json.loads(r'''{
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Host": "www.dogforum.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent":"%s"
    }'''%ua.random)
    random_header["Referer"] = url
    return random_header

In [125]:
def board_urls(board_name,last_page):
    thread = {}
    for page in range(1,last_page+1):
        url = "http://www.dogforum.com/{}/{}/".format(board_name, page)
        resp = requests.get(url, headers=random_header(url))
        soup = BeautifulSoup(resp.text, 'lxml')
        for num in range(1,250):
            try:
                key = soup.select('table')[11].findAll("a", id=re.compile("^thread_title_"))[num].text
                value = soup.select('table')[11].findAll("a", id=re.compile("^thread_title_"))[num]['href']
                thread[key] = value
            except IndexError:
                break
    threads = list(thread.values())
    content_page_url(threads)

In [139]:
def content_page_url(threads):
    all_urls = []
    for url in threads:
        resp = requests.get(url, headers = random_header(url))
        if not re.findall("Page 1 of (\d*)", resp.text):
            get_content(url)
        else:
            last_page = int(re.findall("Page 1 of (\d*)", resp.text)[0])
            all_urls.append(url)
            urls = [url+'page%s/'%page for page in range(2, last_page+1)]
            all_urls.append(urls)

In [153]:
def get_content(url):
    resp = requests.get(url, headers=random_header(url))
    resp.encoding = 'cp1252'
    soup = BeautifulSoup(resp.text, 'lxml')

    block = {}
    post_date = {}

    for num in range(1,11):
        try:
            block['title'] = soup.find('title').text
            block['story'] = soup.select('.tborder.vbseo_like_postbit')[num].findAll(id=re.compile("^postcount"))[0].get('name')
            block['author_url'] = soup.select('.tborder.vbseo_like_postbit')[num].select('a.bigusername')[0]['href']
            block['author_name'] = soup.select('.tborder.vbseo_like_postbit')[num].select('a.bigusername')[0].text

            data = {}
            data['date'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[0]
            data['time'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[1]
            data['ampm'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[2]
            block['post_date'] = data

            author = {}
            try:
                author['member_type'] = soup.select('.tborder.vbseo_like_postbit')[num].select('div.smallfont')[0].text
            except IndexError:
                pass
            try:
                author['join_date'] = re.findall('Join Date:(.*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
            except IndexError:
                pass
            try:
                author['posts'] = re.findall('Posts: (.*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
            except IndexError:
                pass
            try:
                author['mentioned'] = re.findall('Mentioned: (.*)Tag', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
            except IndexError:
                pass
            try:
                author['tagged'] = re.findall('Tagged: (\d*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip() + ' thread(s)'
            except IndexError:
                pass
            block['author_info'] = author

            a = soup.select('.tborder.vbseo_like_postbit')[num].find("div", id=re.compile("^post_message_"))
            if len(a.select('div')) > 0:
                a.select('div')[0].extract()
                content = a.text
            else:
                content = soup.select('.tborder.vbseo_like_postbit')[num].find("div", id=re.compile("^post_message_")).text
            block['content'] = content.strip().replace("\r","").replace("\n","")
            print(block)
        except IndexError:
            break

In [154]:
board_urls('dog-health',1)

{'title': ' 20 Crate Rest Activities', 'story': '2', 'author_url': 'http://www.dogforum.com/members/10075/', 'author_name': 'efinley', 'post_date': {'date': '06-20-2013,', 'time': '05:03', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Jan 2012 Location: NY', 'posts': '1,262', 'mentioned': '6 Post(s)', 'tagged': '0 thread(s)'}, 'content': "That's awesome"}
{'title': ' 20 Crate Rest Activities', 'story': '3', 'author_url': 'http://www.dogforum.com/members/3987/', 'author_name': 'crock', 'post_date': {'date': '06-20-2013,', 'time': '06:12', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Oct 2010 Location: NW', 'posts': '12,336', 'mentioned': '339 Post(s)', 'tagged': '0 thread(s)'}, 'content': "Zoe, this should be a sticky!  Somewhere on here, there's another crate rest activity video.  I'll post it if I can find it!"}
{'title': ' 20 Crate Rest Activities', 'story': '4', 'author_url': 'http://www.dogforum.com/members/14588/', 'au

{'title': ' Dog 45 days pregnant with a low temp', 'story': '2', 'author_url': 'http://www.dogforum.com/members/487/', 'author_name': 'animalcraker', 'post_date': {'date': 'Today,', 'time': '03:17', 'ampm': 'AM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Mar 2009 Location: So Cal', 'posts': '664', 'mentioned': '11 Post(s)', 'tagged': '0 thread(s)'}, 'content': "Your vet is going to be the best place to get information directly related to your pet and her current health status. 100.0 is within the normal temperature range for dogs, and a single reading in regards to pregnant dogs is not very useful for determining anything with any kind of certainty. How often are you taking her temp? And how are you taking it?You said she's 45 days pregnant, is that post breeding or post ovulation? And how often was she bred?Have you had an ultrasound done to confirm a pregnancy since she's too early for X-rays at this point?How large is she? How large is the stud that you bred her

{'title': ' Stroke?', 'story': '2', 'author_url': 'http://www.dogforum.com/members/136857/', 'author_name': 'Laco', 'post_date': {'date': '03-03-2018,', 'time': '12:14', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Dec 2015 Location: Southern California', 'posts': '1,502', 'mentioned': '21 Post(s)', 'tagged': '0 thread(s)'}, 'content': 'Seizure perhaps, but a visit to the Vet would IMO be in order, to run some tests to attempt to determine cause and possible treatment plan.'}
{'title': ' Any idea the type of rash on the tip of my dogs ear?', 'story': '2', 'author_url': 'http://www.dogforum.com/members/487/', 'author_name': 'animalcraker', 'post_date': {'date': '02-27-2018,', 'time': '08:03', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Mar 2009 Location: So Cal', 'posts': '664', 'mentioned': '11 Post(s)', 'tagged': '0 thread(s)'}, 'content': 'That is definitely not normal and needs to be properly diagnosed by a vet.'}
{'ti

{'title': ' Dog pees/poops only once per day', 'story': '2', 'author_url': 'http://www.dogforum.com/members/136857/', 'author_name': 'Laco', 'post_date': {'date': '02-26-2018,', 'time': '04:31', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Dec 2015 Location: Southern California', 'posts': '1,502', 'mentioned': '21 Post(s)', 'tagged': '0 thread(s)'}, 'content': "She is probably very nervous and trying to figure out where she is, and what happened to her. The pooping once a day shouldn't be a problem, as long as she continues to do that, though most dogs I know do go two or three times a day. The peeing only once a day, could over time become problematic, like leading to a UTI, but, probably taking her out more often, might start conveying the idea to her. Also, if she pees inside, she gets nothing, outside leads to tons of praise and a treat. Won't take her long to figure out, 'I can pee outside for snacks'. When we were training Samantha, we took her out

{'title': ' My dog might be pregnant help!!!!', 'story': '2', 'author_url': 'http://www.dogforum.com/members/209826/', 'author_name': 'Mary1234', 'post_date': {'date': '02-18-2018,', 'time': '05:14', 'ampm': 'PM'}, 'author_info': {'member_type': 'Junior Member', 'join_date': 'Feb 2018', 'posts': '4', 'mentioned': '0 Post(s)', 'tagged': '0 thread(s)'}, 'content': '*Her spayed'}
{'title': ' My dog might be pregnant help!!!!', 'story': '3', 'author_url': 'http://www.dogforum.com/members/209826/', 'author_name': 'Mary1234', 'post_date': {'date': '02-18-2018,', 'time': '05:48', 'ampm': 'PM'}, 'author_info': {'member_type': 'Junior Member', 'join_date': 'Feb 2018', 'posts': '4', 'mentioned': '0 Post(s)', 'tagged': '0 thread(s)'}, 'content': 'Can somebody who has experience with dog breeds and breeding please reply to my thread I’ve been extremely stressed about my poor dog she’s one of the only girl dogs I have ever owned and I have no experience'}
{'title': ' My dog might be pregnant help!!

...

{'title': ' New here. Questions about post surgery', 'story': '2', 'author_url': 'http://www.dogforum.com/members/188377/', 'author_name': 'Shadowmom', 'post_date': {'date': '12-10-2017,', 'time': '01:35', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Jun 2017 Location: Cambridge MA', 'posts': '619', 'mentioned': '30 Post(s)', 'tagged': '0 thread(s)'}, 'content': "My cat had obstruction surgery last summer after my dog chewed up a toy and the cat ate some of the stuffing that was all over the floor before I could clean it up.  Same thing.  He also had a good recovery and threw up a week or or two later.  He's long-haired and does throw up once a month or two months.  He refused to eat ant special diet to rule out I'd or food allergies.  And has seemed fine since but continues to occasionally throw up, once a month or less. The important thing is if it's just once and not constant, how they're acting overall, and if the vomit is clear or just they're food 

{'title': ' Dog has a 360° ring around paw', 'story': '2', 'author_url': 'http://www.dogforum.com/members/200034/', 'author_name': 'Mk71681', 'post_date': {'date': '10-17-2017,', 'time': '03:30', 'ampm': 'PM'}, 'author_info': {'member_type': 'Junior Member', 'join_date': 'Oct 2017', 'posts': '2', 'mentioned': '1 Post(s)', 'tagged': '0 thread(s)'}, 'content': 'My dog has a 360° ring around his paw and cant put any pressure on his paw. Has anyone seen something like this before? Im taking him to the get but not until later tonight. Was wondering if there is anything I can do to help him now'}
{'title': ' Dog has a 360° ring around paw', 'story': '3', 'author_url': 'http://www.dogforum.com/members/193841/', 'author_name': 'gattymu', 'post_date': {'date': '10-20-2017,', 'time': '05:03', 'ampm': 'PM'}, 'author_info': {'member_type': 'Member', 'join_date': 'Aug 2017 Location: Barcelona', 'posts': '67', 'mentioned': '2 Post(s)', 'tagged': '0 thread(s)'}, 'content': "¿Have you got any clue of 

{'title': ' Most Read: 7 Dogs Breeds With Sensitive Souls...', 'story': '2', 'author_url': 'http://www.dogforum.com/members/195898/', 'author_name': 'Markie', 'post_date': {'date': '12-03-2017,', 'time': '10:05', 'ampm': 'AM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Sep 2017', 'posts': '1,485', 'mentioned': '74 Post(s)', 'tagged': '1 thread(s)'}, 'content': 'Interesting stuff!!!  I figures American Pitbull Terrier would be in there. They really are gentle. Australian Shepherds just love people haha   Sent from my Pixel using Tapatalk'}
{'title': ' Help Remembering Vet Appointments', 'story': '2', 'author_url': 'http://www.dogforum.com/members/136857/', 'author_name': 'Laco', 'post_date': {'date': '12-01-2017,', 'time': '07:33', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Dec 2015 Location: Southern California', 'posts': '1,502', 'mentioned': '21 Post(s)', 'tagged': '0 thread(s)'}, 'content': 'How about calendaring it on your smart

{'title': ' Is my dog overweight?', 'story': '2', 'author_url': 'http://www.dogforum.com/members/147826/', 'author_name': 'Chas', 'post_date': {'date': '11-27-2017,', 'time': '09:49', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Mar 2016 Location: Australia', 'posts': '1,255', 'mentioned': '117 Post(s)', 'tagged': '1 thread(s)'}, 'content': "She's pet weight, that's fine. There's a waist. I wouldn't allow her to put on any more. Are you aware that puppies need less food as they mature? Keep an eye on what the packaging says. Also Puppies go through stages of pudgy and thin so if you change things do it slowly."}
{'title': ' Is my dog overweight?', 'story': '3', 'author_url': 'http://www.dogforum.com/members/195898/', 'author_name': 'Markie', 'post_date': {'date': '11-27-2017,', 'time': '09:54', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Sep 2017', 'posts': '1,485', 'mentioned': '74 Post(s)', 'tagged': '1 thread(s)'}, 'co

{'title': ' Dog had hives and now vomiting', 'story': '2', 'author_url': 'http://www.dogforum.com/members/200562/', 'author_name': 'Olympia', 'post_date': {'date': '11-22-2017,', 'time': '02:36', 'ampm': 'PM'}, 'author_info': {'member_type': 'Senior Member', 'join_date': 'Oct 2017', 'posts': '136', 'mentioned': '6 Post(s)', 'tagged': '0 thread(s)'}, 'content': 'How is your dog doing? Did you call the vet?'}
{'title': ' Loud Stomach Noises, Should I be worried?', 'story': '2', 'author_url': 'http://www.dogforum.com/members/174842/', 'author_name': 'Lakerfanalways', 'post_date': {'date': '11-20-2017,', 'time': '02:18', 'ampm': 'PM'}, 'author_info': {'member_type': 'Member', 'join_date': 'Dec 2016', 'posts': '49', 'mentioned': '0 Post(s)', 'tagged': '0 thread(s)'}, 'content': 'If I were you to be on the safe side I would take her to the Vet..especially since its Thanksgiving week this week so many Vets will be unavailable..just take her in to be on the safe side..the fact that she made tw